In [16]:
import csv
import pandas as pd

In [19]:
from mysql.connector import MySQLConnection, Error

In [21]:
#Making connection to SQL server
def make_connection():
    try:
        conn = MySQLConnection(host='IES-ADS-ClassDB.sjsu.edu',
                               database='querycrew_db',
                               user='querycrew_user',
                               password='Pomegranate_746')
        
        if conn.is_connected():
            print('Connected to the MySQL database!')
            
            return conn
                
    except Error as e:
        print('Connection failed.')
        print(e)
        
        return None

In [23]:
conn = make_connection()
cursor = conn.cursor()
#Up to here code runs smooth

Connected to the MySQL database!


In [ ]:
#what happened is my code was running fine, however, when I got to this part of the 
#CREATING LOCATION TABLE
cursor.execute('DROP TABLE IF EXISTS location')

In [ ]:
cursor.execute('DROP TABLE IF EXISTS vehicle')

In [14]:
#This is for the one-to-one tables. The primary key is VIN(1-10) 
#CREATING VEHICLE TABLE

sql = ( """
        CREATE TABLE vehicle
        (
            vin varchar(32),
            make varchar(32),
            model varchar(32),
            model_year int,
            PRIMARY KEY(vin)
        )
        """
      )
cursor.execute(sql)

KeyboardInterrupt: 

In [ ]:
sql = ( """
        CREATE TABLE location
        (
            vin varchar(20),
            county varchar(32),
            city varchar(32),
            state varchar(32),
            zip_code int,
            FOREIGN KEY (vin) REFERENCES vehicle(vin)
        )
        """
      )

cursor.execute(sql);

In [ ]:
#For these two tables the primary key is the vin, and the foriegn key is in the location table and it is the vin as well

In [ ]:
#def remove_duplicates(

In [ ]:
sql_insert_vehicle = (  
    """
    INSERT INTO vehicle (vin, model_year, make, model)
    VALUES (%s, %s, %s, %s)
    """
)

sql_insert_location = ( 
    """
    INSERT INTO location (vin, county, city, state, zip_code)
    VALUES (%s, %s, %s, %s, %s)
    """
)

In [ ]:
def transform_vehicle(row):
    #Converting the model_year column and zip-code column to an int
    try:
      row[1] = int(row[1])
    except ValueError:
      row[4] = None

    return row

In [ ]:
def transform_location(row):
    #Converting the model_year column and zip-code column to an int
    try:
      row[4] = int(row[4])
    except ValueError:
      row[4] = None

    return row

In [ ]:
#This works, all you need to do is create a new table with the new variable sizes and run it again
vehicle_cols = [0, 5, 6, 7]
location_cols = [0, 1, 2, 3, 4]
first = True
seen_vins = set()  

# Read CSV data into a list before processing
with open('Electric_Vehicle_Population_Data.csv', newline='') as csv_file:
    data = list(csv.reader(csv_file, delimiter=',', quotechar='"'))  

for row in data:
    if not first:
        vin = row[vehicle_cols[0]]  

        # Skips over the duplicates found in the file
        if vin in seen_vins:
            continue  
        seen_vins.add(vin)  

        # Checking if the VIN already exists in the database
        cursor.execute("SELECT COUNT(*) FROM vehicle WHERE vin = %s", (vin,))
        if cursor.fetchone()[0] > 0:
            continue  # Skip database duplicates too

        vehicle_row = [row[i] for i in vehicle_cols]
        #print(vehicle_row)  

        # Insert into vehicle table
        cursor.execute(sql_insert_vehicle, vehicle_row)

    first = False  

conn.commit()


In [ ]:
sql = """
    SELECT * FROM vehicle LIMIT 25
"""

cursor.execute(sql)
results = cursor.fetchall()

for row in results:
    print(row)

conn.commit()

In [19]:
cursor.close()
conn.close()